Setup environment and load the base PyPSA-Earth network for a specified country.


In [ ]:

import os
import sys
import warnings
import pypsa
import warnings
import pypsa
from pathlib import Path
import os
import copy
from os.path import join
import pandas as pd
import geopandas as gpd
import requests
import shutil
# Import all dirs
parent_dir = Path(os.getcwd()).parents[0]
sys.path.append(str(parent_dir))
from src.paths import all_dirs

dirs = all_dirs()

import logging
from pathlib import Path
# Add PyPSA-Earth scripts to the system path
scripts_path = os.path.join(parent_dir.parents[0], "pypsa-earth", "scripts")
assert os.path.isdir(scripts_path), f"Path not found: {scripts_path}"

sys.path.append(scripts_path)



LOG_FILE = join(parent_dir, "logs.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(LOG_FILE, encoding="utf-8")],
)


# Suppress warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)
# Suppress unnecessary warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

warnings.filterwarnings("ignore")
# Load the base network file path
network_dir = dirs["data/processed/networks"]
network_files = [
    "network_original",
    "network_snapped",
    "network_expanded",
    "network_expanded_no_orphans",
    "network_nuclear",
    "network_prod_mix",
    "network_base",
]

networks_dict = {
    nf: pypsa.Network(join(network_dir, f"{nf}.nc")) for nf in network_files
}



In [ ]:

ppl_file = join(dirs["data/processed/generation"],
                "powerplants_all.csv")
ppl = pd.read_csv(ppl_file, index_col=0)
# Update ppl to match pypsa network

# Load config file
ppmatching = os.path.join(
    Path(os.getcwd()).parents[1],
      "pypsa-earth", "configs", "powerplantmatching_config.yaml"
)
print(ppmatching)
config = pm.get_config(ppmatching)

# Select target countries
config["target_countries"] = ["EC"]

import powerplantmatching as pm

ppl_file = join(dirs["data/processed/generation"],
                "powerplants_all.csv")
ppl = pd.read_csv(ppl_file, index_col=0)
# Update ppl to match pypsa network

# Load config file
ppmatching = os.path.join(
    Path(os.getcwd()).parents[1],
      "pypsa-earth", "configs", "powerplantmatching_config.yaml"
)
print(ppmatching)
config = pm.get_config(ppmatching)

# Select target countries
config["target_countries"] = ["EC"]

# For the base case, I take the 2025 trend scenario with linear growt
path_loads = dirs["data/processed/scaled_loads"]
load_profile_name = "load_base_2030_linear.csv"
load_profile_file = join(path_loads, load_profile_name)

load_profile = pd.read_csv(load_profile_file, index_col=0, parse_dates=True)


In [ ]:

network = copy.deepcopy(networks_dict["network_base"])

# Add the load profile to the network
network.madd(
    "Load", load_profile.columns, bus=load_profile.columns, p_set=load_profile
)
#network_base.loads_t.p_set
# Check peak load (rounded to 1 decimal place)
peak_load = round(network.loads_t.p_set.T.sum().max() / 1000, 1)
logging.info(f"Peak load: {peak_load} GW")  # GW# loads
# For the base case, I take the 2025 trend scenario with linear growt
path_loads = dirs["data/processed/scaled_loads"]
load_profile_name = "load_base_2030_linear.csv"
load_profile_file = join(path_loads, load_profile_name)

load_profile = pd.read_csv(load_profile_file, index_col=0, parse_dates=True)
# Add the load profile to the network
network.madd(
    "Load", load_profile.columns, bus=load_profile.columns, p_set=load_profile
)
#network_base.loads_t.p_set
# Check peak load (rounded to 1 decimal place)
peak_load = round(network.loads_t.p_set.T.sum().max() / 1000, 1)
logging.info(f"Peak load: {peak_load} GW")  # GW


In [ ]:
ppl_p = ppl_f.powerplant.to_pypsa_names()
ppl_p = ppl_p.dropna(axis=1, how="all")

ppl_p["p_nom"] = pd.to_numeric(ppl_p["p_nom"], errors="coerce")


# Ensure capacity numeric (handles things like "1'500,00")
ppl_p["p_nom"] = (
    ppl_p["p_nom"].astype(str)
    .str.replace("'", "", regex=False)
    .str.replace(",", ".", regex=False)
)
ppl_p["p_nom"] = pd.to_numeric(ppl_p["p_nom"], errors="coerce").fillna(0.0)


import unicodedata
import re
import pandas as pd


import unicodedata
import re
import pandas as pd

# 1) Helper to normalize strings: trim, lowercase, remove accents, collapse spaces
def _normalize(s):
    if pd.isna(s):
        return None
    s = str(s).strip().lower()
    # remove accents
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    # collapse internal whitespace
    s = re.sub(r"\s+", " ", s)
    return s

# 2) Normalize the Spanish carriers to a helper column
ppl_p["carrier_norm"] = ppl_p["carrier"].apply(_normalize)

# 3) Your mapping (keys should be normalized/without accents/lowercase)
carrier_map_es2en = {
    "hidraulica": "Hydro",
    "hidroelectrico": "Hydro",
    "termica": "Thermal",
    "termoelectrico": "Thermal",
    "biomasa": "Biomass",
    "fotovoltaica": "Solar PV",
    "eolica": "Wind",
    "eolico": "Wind",
    "biogas": "Biogas",
    "ernc": "Other RE", # or "Other RE" if you prefer  
    "nuclear": "Nuclear"
}

# 4) Map to English (new column). Keep original untouched.
ppl_p["carrier"] = ppl_p["carrier_norm"].map(carrier_map_es2en)

# 5) (Optional) If you want to overwrite the original column instead:
# ppl_p["carrier"] = ppl_p["carrier_en"].fillna(ppl_p["carrier"])

# 6) (Optional) Inspect what didnâ€™t map (so you can extend the dictionary)
unmapped = (
    ppl_p.loc[ppl_p["carrier"].isna() & ppl_p["carrier_norm"].notna(), "carrier_norm"]
    .value_counts()
)


In [ ]:

n = 0

for i in ppl_filtered.index:
    pp = ppl_filtered.loc[[i]]
    network.madd("Generator", pp.index, **pp)
    n+=1
    k = len(network.generators)
    if n!= k:
        print("not working for ", i)
        n-=1

#network.generators#Safe way to add the plant and detect duplicates
n = 0

for i in ppl_filtered.index:
    pp = ppl_filtered.loc[[i]]
    network.madd("Generator", pp.index, **pp)
    n+=1
    k = len(network.generators)
    if n!= k:
        print("not working for ", i)
        n-=1



# Filter Power Plants by size and by conection to network
FILTERING_POWER = 10
DATE_IN = 2017
ppl_connected = ppl_p[ppl_p['component']=="S.N.I."]  # Keep only plants connected to the network
ppl_connected_current = ppl_connected[ppl_connected['DateIn']<=DATE_IN]  # Example: filter plants commissioned before or in 2024
#ppl_connected_future = ppl_connected[ppl_connected['DateIn']>2017]  # Example: filter plants commissioned before or in 2024
ppl_filtered = ppl_connected_current[ppl_connected_current['p_nom']>=FILTERING_POWER]  # Example: filter plants with capacity >= 100 kW

ppl_filtered# Filter Power Plants by size and by conection to network
FILTERING_POWER = 10
DATE_IN = 2017
ppl_connected = ppl_p[ppl_p['component']=="S.N.I."]  # Keep only plants connected to the network
ppl_connected_current = ppl_connected[ppl_connected['DateIn']<=DATE_IN]  # Example: filter plants commissioned before or in 2024
#ppl_connected_future = ppl_connected[ppl_connected['DateIn']>2017]  # Example: filter plants commissioned before or in 2024
ppl_filtered = ppl_connected_current[ppl_connected_current['p_nom']>=FILTERING_POWER]  # Example: filter plants with capacity >= 100 kW

ppl_filtered


In [ ]:


from scipy.spatial import cKDTree as KDTree
import numpy as np
import matplotlib.pyplot as plt



substation_i = network.buses.query("substation_lv").index
kdtree = KDTree(network.buses.loc[substation_i, ["x", "y"]].values)
ppl_i = ppl_filtered.index

tree_i = kdtree.query(ppl_filtered.loc[ppl_i, ["lon", "lat"]].values)[1]
ppl_filtered.loc[ppl_i, "bus"] = substation_i.append(pd.Index([np.nan]))[tree_i]

# Bus needs to be string apparently
ppl_filtered["bus"] = ppl_filtered["bus"].astype(str)
missing = ~ppl_filtered["bus"].isin(network.buses.index)
print(ppl_filtered[missing][["bus"]].head()) # should be emptyfrom scipy.spatial import cKDTree as KDTree
import numpy as np
import matplotlib.pyplot as plt

substation_i = network.buses.query("substation_lv").index
kdtree = KDTree(network.buses.loc[substation_i, ["x", "y"]].values)
ppl_i = ppl_filtered.index

tree_i = kdtree.query(ppl_filtered.loc[ppl_i, ["lon", "lat"]].values)[1]
ppl_filtered.loc[ppl_i, "bus"] = substation_i.append(pd.Index([np.nan]))[tree_i]

# Bus needs to be string apparently
ppl_filtered["bus"] = ppl_filtered["bus"].astype(str)
missing = ~ppl_filtered["bus"].isin(network.buses.index)
print(ppl_filtered[missing][["bus"]].head()) 

total_capacity = ppl_p['p_nom'].sum()
total_connected = ppl_connected['p_nom'].sum()
total_connected_current = ppl_connected_current['p_nom'].sum()
total_filtered = ppl_filtered['p_nom'].sum()
logging.info(f"\nTotal capacity: {total_capacity} kW,\nConnected capacity: {total_connected} kW, \
             \nConnected current capacity: {total_connected_current} kW, \
             \nFiltered capacity: {total_filtered} kW")  
total_capacity = ppl_p['p_nom'].sum()
total_connected = ppl_connected['p_nom'].sum()
total_connected_current = ppl_connected_current['p_nom'].sum()
total_filtered = ppl_filtered['p_nom'].sum()
logging.info(f"\nTotal capacity: {total_capacity} kW,\nConnected capacity: {total_connected} kW, \
             \nConnected current capacity: {total_connected_current} kW, \
             \nFiltered capacity: {total_filtered} kW") 


In [ ]:
# Plot the shapefile boundaries on the same axis

# Where to put / find GADM (expects dirs["data/raw/gadm"])
COUNTRY_ISO3 = "ECU"
GADM_BASE_DIR = None  # will be inferred from dirs dict below
GADM_VERSION = "4.1"
GADM_FILE_STEM = f"gadm41_{COUNTRY_ISO3}"
GADM_LAYER_L1 = "ADM_ADM_1"
GADM_URL = f"https://geodata.ucdavis.edu/gadm/gadm{GADM_VERSION}/gpkg/{GADM_FILE_STEM}.gpkg"

def ensure_gadm_file(gadm_path: str) -> str:
    """Ensure GADM gpkg exists locally; download if missing. Return path to gpkg."""
    gpkg_path = os.path.join(gadm_path, f"{GADM_FILE_STEM}.gpkg")
    gpkg_dir = Path(gpkg_path).parent
    if not Path(gpkg_path).is_file():
        gpkg_dir.mkdir(parents=True, exist_ok=True)
        resp = requests.get(GADM_URL, stream=True, timeout=300)
        resp.raise_for_status()
        with open(gpkg_path, "wb") as f:
            shutil.copyfileobj(resp.raw, f)
    else:
        print(f"GADM file already exists: {gpkg_path}")
    assert Path(gpkg_path).is_file(), f"GADM file not found or failed to download: {gpkg_path}"
    return gpkg_path

GADM_inputfile_gpkg = ensure_gadm_file(dirs["data/raw/gadm"])

shapefile = gpd.read_file(GADM_inputfile_gpkg, layer='ADM_ADM_1')
#shapefile.boundary.plot(ax=ax, linewidth=0.2, color='black')#need to load the shapefile before nodes
# Plot the shapefile boundaries on the same axis

# Where to put / find GADM (expects dirs["data/raw/gadm"])
COUNTRY_ISO3 = "ECU"
GADM_BASE_DIR = None  # will be inferred from dirs dict below
GADM_VERSION = "4.1"
GADM_FILE_STEM = f"gadm41_{COUNTRY_ISO3}"
GADM_LAYER_L1 = "ADM_ADM_1"
GADM_URL = f"https://geodata.ucdavis.edu/gadm/gadm{GADM_VERSION}/gpkg/{GADM_FILE_STEM}.gpkg"

def ensure_gadm_file(gadm_path: str) -> str:
    """Ensure GADM gpkg exists locally; download if missing. Return path to gpkg."""
    gpkg_path = os.path.join(gadm_path, f"{GADM_FILE_STEM}.gpkg")
    gpkg_dir = Path(gpkg_path).parent
    if not Path(gpkg_path).is_file():
        gpkg_dir.mkdir(parents=True, exist_ok=True)
        resp = requests.get(GADM_URL, stream=True, timeout=300)
        resp.raise_for_status()
        with open(gpkg_path, "wb") as f:
            shutil.copyfileobj(resp.raw, f)
    else:
        print(f"GADM file already exists: {gpkg_path}")
    assert Path(gpkg_path).is_file(), f"GADM file not found or failed to download: {gpkg_path}"
    return gpkg_path

GADM_inputfile_gpkg = ensure_gadm_file(dirs["data/raw/gadm"])

shapefile = gpd.read_file(GADM_inputfile_gpkg, layer='ADM_ADM_1')
#shapefile.boundary.plot(ax=ax, linewidth=0.2, color='black')




In [ ]:

# Add cost data from 'costs' to 'network.generators'
network.generators.drop(['marginal_cost', 'capital_cost'], axis=1, inplace=True)

network.generators = pd.merge(network.generators, costs, left_on='carrier', right_index=True)
network.generators[['bus', 'p_nom', 'carrier', 'marginal_cost', 'capital_cost']]
network.generators.drop(['marginal_cost', 'capital_cost'], axis=1, inplace=True)

# Add cost data from 'costs' to 'network.generators'
network.generators = pd.merge(network.generators, costs, left_on='carrier', right_index=True)
network.generators[['bus', 'p_nom', 'carrier', 'marginal_cost', 'capital_cost']]

 # filename = 'resources/' + country_code + '/costs_nuclear.csv' # The CSV file has been created with PyPSA-Earth
cost_filename = join(dirs["data/raw/generation"], "technology_cost.csv")
# TODO: Calculate annuity for capital cost, check marginal cost with variable maintenance
costs = pd.read_csv(
    cost_filename,
    index_col=0,
    comment = "#"
)# filename = 'resources/' + country_code + '/costs_nuclear.csv' # The CSV file has been created with PyPSA-Earth
cost_filename = join(dirs["data/raw/generation"], "technology_cost.csv")
# TODO: Calculate annuity for capital cost, check marginal cost with variable maintenance
costs = pd.read_csv(
    cost_filename,
    index_col=0,
    comment = "#"
)

from shapely.geometry import Point
# Attach node to region
def attach_node_to_region(gdf, longitude, latitude):
    # Define the point (longitude, latitude)
    point = Point(longitude, latitude)
    # Ensure the CRS of GeoDataFrame and point match
    if gdf.crs != 'EPSG:4326': # Assuming the point is in WGS84 (EPSG:4326)
        gdf = gdf.to_crs('EPSG:4326')
    # Find the polygon that contains the point and get the coordinates of the centroid
    region = gdf[gdf.contains(point)]['NAME_1'].values[0]
    region_x = gdf[gdf.contains(point)]['x'].values[0]
    region_y = gdf[gdf.contains(point)]['y'].values[0]
    return region, region_x, region_y


In [ ]:


# Attach node to region
def attach_node_to_region(gdf, longitude, latitude):
    # Define the point (longitude, latitude)
    point = Point(longitude, latitude)
    # Ensure the CRS of GeoDataFrame and point match
    if gdf.crs != 'EPSG:4326': # Assuming the point is in WGS84 (EPSG:4326)
        gdf = gdf.to_crs('EPSG:4326')
    # Find the polygon that contains the point and get the coordinates of the centroid
    region = gdf[gdf.contains(point)]['NAME_1'].values[0]
    region_x = gdf[gdf.contains(point)]['x'].values[0]
    region_y = gdf[gdf.contains(point)]['y'].values[0]
    return region, region_x, region_y

def get_node_by_region(region_node):
    """Consider each region as a node.
    The centroid of the polygon for regions is the position of the node.

    Important:
    The projection of the maps is ``EPSG:4674 (SIRGAS 2000) <https://epsg.io/4674>``, unit- degree, geographic CRS.
    To use calculations, e.g, "centroid", "sjoin",  the map needs to be reprojected, ``.to_crs('epsg:4087')``

    EPSG:4326 is also a geographic CRS, unit - degree,
    # issue: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.
    # previous: to_crs('epsg:4326'), now: set .to_crs('epsg:4087') solved: see https://gis.stackexchange.com/questions/372564/userwarning-when-trying-to-get-centroid-from-a-polygon-geopandas

    """

    region_node['x'] = region_node.centroid.x
    region_node['y'] = region_node.centroid.y
    return region_node

region_node = get_node_by_region(shapefile)
#region_node.to_csv("regions.csv")



In [ ]:

# Load the cutout
cutout_file = join(dirs["data/raw/cutouts"], "cutout-2013-era5.nc")
cutout = atlite.Cutout(cutout_file)import atlite

# Load the cutout
cutout_file = join(dirs["data/raw/cutouts"], "cutout-2013-era5.nc")
cutout = atlite.Cutout(cutout_file)

 # Cluster the network based on the busmap
from pypsa.clustering.spatial import get_clustering_from_busmap
clustered = get_clustering_from_busmap(network, busmap).network
clustered.generators.to_csv("clustered_gens.csv")

# Cluster the network based on the busmap
from pypsa.clustering.spatial import get_clustering_from_busmap
clustered = get_clustering_from_busmap(network, busmap).network
clustered.generators.to_csv("clustered_gens.csv")ƒx°X ƒ{ƒ{ õ'network.lines = network.lines.reindex(columns=network.components['Line']['attrs'].index[1:])
network.lines['type'] = np.nan
network.buses = network.buses.reindex(columns=network.components['Bus']['attrs'].index[1:])
network.buses['frequency'] = 60network.lines = network.lines.reindex(columns=network.components['Line']['attrs'].index[1:])
network.lines['type'] = np.nan
network.buses = network.buses.reindex(columns=network.components['Bus']['attrs'].index[1:])
network.buses['frequency'] = 60L°W OO õ&busmap = attach_region_to_buses(region_node, network.buses)['region']
busmap.to_csv("bus_mapped")busmap = attach_region_to_buses(region_node, network.buses)['region']
busmap.to_csv("bus_mapped")‰
°V ‰
‰
 õ%# Attach region to buses using the attach_node_to_region function
def attach_region_to_buses(gdf, buses):
    buses['region'] = 'NA'
    for i in buses.index:
        lon = buses.loc[i]['lon']
        lat = buses.loc[i]['lat']
        try:
            region, region_x, region_y = attach_node_to_region(gdf, lon, lat)
        except Exception as e:
            print('Error:', e)
            print('Index:', i)
            continue
        buses.loc[i, 'region'] = region
        buses.loc[i, 'region_x'] = region_x
        buses.loc[i, 'region_y'] = region_y
    return buses# Attach region to buses using the attach_node_to_region function
def attach_region_to_buses(gdf, buses):
    buses['region'] = 'NA'
    for i in buses.index:
        lon = buses.loc[i]['lon']
        lat = buses.loc[i]['lat']
        try:
            region, region_x, region_y = attach_node_to_region(gdf, lon, lat)
        except Exception as e:
            print('Error:', e)
            print('Index:', i)
            continue
        buses.loc[i, 'region'] = region
        buses.loc[i, 'region_x'] = region_x
        buses.loc[i, 'region_y'] = region_y
    return buses  ! vectorized as shvec
# from shapely import vectorized as shvec  # works when available

# # 0) Mainland polygon (exclude GalÃ¡pagos by longitude)
# shp_geo = shapefile.to_crs(4326)
# continental = shp_geo[shp_geo.geometry.centroid.x > -85]
# mainland_geom = unary_union(continental.geometry)

# # 1) Pick coord names from your preprocessed objects (X,Y are already defined above)
# def prepare_for_plot(da):
#     # wrap longitudes to -180..180 if needed
#     x = da[X].values
#     if x.min() >= 0 and x.max() > 180:
#         x_wrapped = ((x + 180) % 360) - 180
#         order = np.argsort(x_wrapped)
#         da = da.isel({X: order}).assign_coords({X: x_wrapped[order]})
#     # sort so y increases upward and x increases to the right
#     if da[Y][0] > da[Y][-1]:
#         da = da.sortby(Y)
#     if da[X][0] > da[X][-1]:
#         da = da.sortby(X)
#     return da

# wind_da = prepare_for_plot(wnd100m)
# solar_da = prepare_for_plot(influx)


# # --- 2) Build a cell-intersection mask (keeps tiles partially inside) ---
# import numpy as np
# from shapely.ops import unary_union
# from shapely.geometry import box
# import shapely

# # mainland_geom already computed above; wind_da/solar_da already prepared

# # helper: edges from center coordinates (assumes monotonic)
# def edges_from_centers(c):
#     c = np.asarray(c, dtype=float)
#     d = np.diff(c)
#     e = np.empty(c.size + 1, dtype=float)
#     e[1:-1] = c[:-1] + d/2
#     e[0]     = c[0]  - d[0]/2
#     e[-1]    = c[-1] + d[-1]/2
#     return e

# # Use the wind grid (same grid as solar after your coarsen); reuse for both
# x_centers = wind_da[X].values
# y_centers = wind_da[Y].values
# x_edges = edges_from_centers(x_centers)
# y_edges = edges_from_centers(y_centers)

# nx, ny = len(x_centers), len(y_centers)

# # build cell polygons (vectorized if shapely>=2; fallback to list comp)
# # boxes shape: (ny, nx)
# boxes = np.empty((ny, nx), dtype=object)
# for j in range(ny):
#     for i in range(nx):
#         boxes[j, i] = box(x_edges[i], y_edges[j], x_edges[i+1], y_edges[j+1])

# try:
#     # shapely 2.0 vectorized predicate (fast)
#     mask_cells = shapely.intersects(boxes, mainland_geom)
# except Exception:
#     # fallback
#     mask_cells = np.array([[boxes[j, i].intersects(mainland_geom) for i in range(nx)]
#                            for j in range(ny)], dtype=bool)

# # apply mask: keep cells that intersect the mainland polygon
# wind_masked  = np.ma.masked_where(~mask_cells, wind_da.values)
# # solar uses same mask because same grid
# solar_masked = np.ma.masked_where(~mask_cells, solar_da.values)


# # --- build a cell-intersection mask (keeps tiles that touch Ecuador) ---
# from shapely.geometry import box
# from shapely.ops import unary_union
# import shapely
# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

# # mainland polygon (you already computed this above)
# shp_geo = shapefile.to_crs(4326)
# continental = shp_geo[shp_geo.geometry.centroid.x > -85]
# mainland_geom = unary_union(continental.geometry)

# def edges_from_centers(c):
#     c = np.asarray(c, dtype=float)
#     d = np.diff(c)
#     e = np.empty(c.size + 1, dtype=float)
#     e[1:-1] = c[:-1] + d/2
#     e[0]     = c[0]  - d[0]/2
#     e[-1]    = c[-1] + d[-1]/2
#     return e

# # Use wind grid (same as solar after your coarsen)
# x_cent = wind_da[X].values
# y_cent = wind_da[Y].values
# x_ed = edges_from_centers(x_cent)
# y_ed = edges_from_centers(y_cent)
# nx, ny = len(x_cent), len(y_cent)

# # cell polygons & intersects mask
# mask_cells = np.zeros((ny, nx), dtype=bool)
# for j in range(ny):
#     for i in range(nx):
#         cell = box(x_ed[i], y_ed[j], x_ed[i+1], y_ed[j+1])
#         mask_cells[j, i] = cell.intersects(mainland_geom)

# # keep whole-cell values where mask is True, NaN elsewhere
# wind_vals  = np.where(mask_cells, wind_da.values,  np.nan)
# solar_vals = np.where(mask_cells, solar_da.values, np.nan)

# # --- color limits (choose fixed or robust) ---
# w_vmin, w_vmax = 0, 8     # e.g., m/s
# s_vmin, s_vmax = 0, 250    # e.g., W/mÂ² (d  "aily mean-ish)

# # --- PLOT AS FULL TILES with pcolormesh ---
# fig, ax = plt.subplots(1, 2, subplot_kw={'projection': ccrs.PlateCarree()},
#                        figsize=(15, 7), constrained_layout=True)

# pc0 = ax[0].pcolormesh(x_ed, y_ed, wind_vals,
#                        transform=ccrs.PlateCarree(),
#                        vmin=w_vmin, vmax=w_vmax,
#                        shading='flat', rasterized=True)
# ax[0].set_title("Mean Wind Potential in m/s", fontsize=14, pad=20)
# continental.plot(ax=ax[0], edgecolor="k", facecolor="none", linewidth=0.6)
# plt.colorbar(pc0, ax=ax[0], label="m/s")

# pc1 = ax[1].pcolormesh(x_ed, y_ed, solar_vals,
#                        transform=ccrs.PlateCarree(),
#                        vmin=s_vmin, vmax=s_vmax,
#                        shading='flat', rasterized=True)
# ax[1].set_title("Mean Influx in W/mÂ²", fontsize=14)
# continental.plot(ax=ax[1], edgecolor="k", facecolor="none", linewidth=0.6)
# plt.colorbar(pc1, ax=ax[1], label="W/mÂ²")

# # zoom to mainland bounds
# minx, miny, maxx, maxy = continental.total_bounds
# for a in ax:
#     a.set_extent([minx, maxx, miny, maxy], crs=ccrs.PlateCarree())

# plt.show()# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

# geo_crs = 4326
# area_crs = 6933

# # 0) Compute Ecuador-only bounds (you already set them)
# shp_geo = shapefile.to_crs(4326)
# minx, miny, maxx, maxy = shp_geo.total_bounds
# # --- SPEED UPS START HERE ---

# # 1) Pull only needed variables and spatially crop FIRST
# ds = cutout.data[["wnd100m", "influx_direct", "influx_diffuse"]]

# # atlite cutouts usually use x/y coords; fall back to lon/lat if present
# X = "x" if "x" in ds.coords else ("lon" if "lon" in ds.coords else None)
# Y = "y" if "y" in ds.coords else ("lat" if "lat" in ds.coords else None)
# assert X and Y, f"Couldn't find spatial coords. Have: {list(ds.coords)}"

# # Handle ascending/descending coordinate order when slicing
# xslice = slice(minx, maxx) if ds[X][0] < ds[X][-1] else slice(maxx, minx)
# yslice = slice(miny, maxy) if ds[Y][0] < ds[Y][-1] else slice(maxy, miny)
# ds = ds.sel({X: xslice, Y: yslice})

# # 2) Thin the time axis (e.g., take every 6th time step) and cast to float32
# ds = ds.isel(time=slice(None, None, 6)).astype("float32")

# # 3) Coarsen spatially (increase factors if still slow: e.g., 6 or 8)
# ds = ds.coarsen({X: 4, Y: 4}, boundary="trim").mean()

# # 4) Reduce over time ONCE and materialize in memory
# wnd100m = ds["wnd100m"].mean(dim="time").compute()
# influx  = (ds["influx_direct"].mean(dim="time") + ds["influx_diffuse"].mean(dim="time")).compute()


# # --- PLOTTING ONLY: replace everything from "fig, ax = plt.subplots(...)" down ---

# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# from shapely.ops import unary_union
# import shapely
# # if using Shapely <2.0, use: from shapely import vectorized as shvec
# from shapely import vectorized as shvec  # works when available

# # 0) Mainland polygon (exclude GalÃ¡pagos by longitude)
# shp_geo = shapefile.to_crs(4326)
# continental = shp_geo[shp_geo.geometry.centroid.x > -85]
# mainland_geom = unary_union(continental.geometry)

# # 1) Pick coord names from your preprocessed objects (X,Y are already defined above)
# def prepare_for_plot(da):
#     # wrap longitudes to -180..180 if needed
#     x = da[X].values
#     if x.min() >= 0 and x.max() > 180:
#         x_wrapped = ((x + 180) % 360) - 180
#         order = np.argsort(x_wrapped)
#         da = da.isel({X: order}).assign_coords({X: x_wrapped[order]})
#     # sort so y increases upward and x increases to the right
#     if da[Y][0] > da[Y][-1]:
#         da = da.sortby(Y)
#     if da[X][0] > da[X][-1]:
#         da = da.sortby(X)
#     return da

# wind_da = prepare_for_plot(wnd100m)
# solar_da = prepare_for_plot(influx)


# # --- 2) Build a cell-intersection mask (keeps tiles partially inside) ---
# import numpy as np
# from shapely.ops import unary_union
# from shapely.geometry import box
# import shapely

# # mainland_geom already computed a    bove; wind_da/solar_da already prepared

# # helper: edges from center coordinates (assumes monotonic)
# def edges_from_centers(c):
#     c = np.asarray(c, dtype=float)
#     d = np.diff(c)
#     e = np.empty(c.size + 1, dtype=float)
#     e[1:-1] = c[:-1] + d/2
#     e[0]     = c[0]  - d[0]/2
#     e[-1]    = c[-1] + d[-1]/2
#     return e

# # Use the wind grid (same grid as solar after your coarsen); reuse for both
# x_centers = wind_da[X].values
# y_centers = wind_da[Y].values
# x_edges = edges_from_centers(x_centers)
# y_edges = edges_from_centers(y_centers)

# nx, ny = len(x_centers), len(y_centers)

# # build cell polygons (vectorized if shapely>=2; fallback to list comp)
# # boxes shape: (ny, nx)
# boxes = np.empty((ny, nx), dtype=object)
# for j in range(ny):
#     for i in range(nx):
#         boxes[j, i] = box(x_edges[i], y_edges[j], x_edges[i+1], y_edges[j+1])

# try:
#     # shapely 2.0 vectorized predicate (fast)
#     mask_cells = shapely.intersects(boxes, mainland_geom)
# except Exception:
#     # fallback
#     mask_cells = np.array([[boxes[j, i].intersects(mainland_geom) for i in range(nx)]
#                            for j in range(ny)], dtype=bool)

# # apply mask: keep cells that intersect the mainland polygon
# wind_masked  = np.ma.masked_where(~mask_cells, wind_da.values)
# # solar uses same mask because same grid
# solar_masked = np.ma.masked_where(~mask_cells, solar_da.values)


# # --- build a cell-intersection mask (keeps tiles that touch Ecuador) ---
# from shapely.geometry import box
# from shapely.ops import unary_union
# import shapely
# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

# # mainland polygon (you already computed this above)
# shp_geo = shapefile.to_crs(4326)
# continental = shp_geo[shp_geo.geometry.centroid.x > -85]
# mainland_geom = unary_union(continental.geometry)

# def edges_from_centers(c):
#     c = np.asarray(c, dtype=float)
#     d = np.diff(c)
#     e = np.empty(c.size + 1, dtype=float)
#     e[1:-1] = c[:-1] + d/2
#     e[0]     = c[0]  - d[0]/2
#     e[-1]    = c[-1] + d[-1]/2
#     return e

# # Use wind grid (same as solar after your coarsen)
# x_cent = wind_da[X].values
# y_cent = wind_da[Y].values
# x_ed = edges_from_centers(x_cent)
# y_ed = edges_from_centers(y_cent)
# nx, ny = len(x_cent), len(y_cent)

# # cell polygons & intersects mask
# mask_cells = np.zeros((ny, nx), dtype=bool)
# for j in range(ny):
#     for i in range(nx):
#         cell = box(x_ed[i], y_ed[j], x_ed[i+1], y_ed[j+1])
#         mask_cells[j, i] = cell.intersects(mainland_geom)

# # keep whole-cell values where mask is True, NaN elsewhere
# wind_vals  = np.where(mask_cells, wind_da.values,  np.nan)
# solar_vals = np.where(mask_cells, solar_da.values, np.nan)

# # --- color limits (choose fixed or robust) ---
# w_vmin, w_vmax = 0, 8     # e.g., m/s
# s_vmin, s_vmax = 0, 250    # e.g., W/mÂ² (daily mean-ish)

# # --- PLOT AS FULL TILES with pcolormesh ---
# fig, ax = plt.subplots(1, 2, subplot_kw={'projection': ccrs.PlateCarree()},
#                        figsize=(15, 7), constrained_layout=True)

# pc0 = ax[0].pcolormesh(x_ed, y_ed, wind_vals,
#                        transform=ccrs.PlateCarree(),
#                        vmin=w_vmin, vmax=w_vmax,
#                        shading='flat', rasterized=True)
# ax[0].set_title("Mean Wind Potential in m/s", fontsize=14, pad=20)
# continental.plot(ax=ax[0], edgecolor="k", facecolor="none", linewidth=0.6)
# plt.colorbar(pc0, ax=ax[0], label="m/s")

# pc1 = ax[1].pcolormesh(x_ed, y_ed, solar_vals,
#                        transform=ccrs.PlateCarree(),
#                        vmin=s_vmin, vmax=s_vmax,
#                        shading='flat', rasterized=True)
# ax[1].set_title("Mean Influx in W/mÂ²", fontsize=14)
# continental.plot(ax=ax[1], edgecolor="k", facecolor="none", linewidth=0.6)
# plt.colorbar(pc1, ax=ax[1], label="W/mÂ²")

# # zoom to mainland bounds
# minx, miny, maxx, maxy = continental.total_bounds
# for a in ax:
#     a.set_extent([minx, maxx, miny, maxy], crs=ccrs.PlateCarree())

# plt.show()
    Z 	( Z                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              ƒJ°\ ƒMƒM õ+import xarray as xr

# Calculate the area of each cell in the cutout
area = cutout.grid.to_crs(area_crs).area / 1e6
area = xr.DataArray(area.values.reshape(cutout.shape), [cutout.coords["y"], cutout.coords["x"]])

area[0][0]import xarray as xr

# Calculate the area of each cell in the cutout
area = cutout.grid.to_crs(area_crs).area / 1e6
area = xr.DataArray(area.values.reshape(cutout.shape), [cutout.coords["y"], cutout.coords["x"]])

area[0][0]íD°[ íGíG õ*# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

# geo_crs = 4326
# area_crs = 6933

# # 0) Compute Ecuador-only bounds (you already set them)
# shp_geo = shapefile.to_crs(4326)
# minx, miny, maxx, maxy = shp_geo.total_bounds
# # --- SPEED UPS START HERE ---

# # 1) Pull only needed variables and spatially crop FIRST
# ds = cutout.data[["wnd100m", "influx_direct", "influx_diffuse"]]

# # atlite cutouts usually use x/y coords; fall back to lon/lat if present
# X = "x" if "x" in ds.coords else ("lon" if "lon" in ds.coords else None)
# Y = "y" if "y" in ds.coords else ("lat" if "lat" in ds.coords else None)
# assert X and Y, f"Couldn't find spatial coords. Have: {list(ds.coords)}"

# # Handle ascending/descending coordinate order when slicing
# xslice = slice(minx, maxx) if ds[X][0] < ds[X][-1] else slice(maxx, minx)
# yslice = slice(miny, maxy) if ds[Y][0] < ds[Y][-1] else slice(maxy, miny)
# ds = ds.sel({X: xslice, Y: yslice})

# # 2) Thin the time axis (e.g., take every 6th time step) and cast to float32
# ds = ds.isel(time=slice(None, None, 6)).astype("float32")

# # 3) Coarsen spatially (increase factors if still slow: e.g., 6 or 8)
# ds = ds.coarsen({X: 4, Y: 4}, boundary="trim").mean()

# # 4) Reduce over time ONCE and materialize in memory
# wnd100m = ds["wnd100m"].mean(dim="time").compute()
# influx  = (ds["influx_direct"].mean(dim="time") + ds["influx_diffuse"].mean(dim="time")).compute()


# # --- PLOTTING ONLY: replace everything from "fig, ax = plt.subplots(...)" down ---

# import numpy as np
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# from shapely.ops import unary_union
# import shapely
# # if using Shapely <2.0, use: from shapely import   



In [ ]:

datasets# Define renewable resources
resources = [
    {'method': 'wind', 'turbine': 'Vestas_V112_3MW', 'capacity_per_sqkm': 4.6, 'resource': 'onwind'},
    {'method': 'pv', 'panel': 'CSi', 'orientation': 'latitude_optimal', 'capacity_per_sqkm': 2, 'resource': 'solar'},
]
correction_factor = 1

# Create new geodataframe for the capacities
capacities = shapefile
datasets = {}
shape = shapefile.set_index("NAME_1")

# Calculate capacities per resource
# TODO: check the algorithm for correctness and efficiency
for resource in resources:
    method =  resource['method']
    res = resource['resource']
    profile_path = join(dirs["data/processed/generation"], f"{res}.nc")
    if Path(profile_path).is_file():
        print('Profile found:', res)
        ds = xr.open_dataset(profile_path)
        profile = ds['profile']
        capacity = ds['capacities']
        capacities[method] = capacity
        datasets[res] = ds
    else:
        print('Profile not found:', res)
        cap_per_sqkm = resource['capacity_per_sqkm']
        print("method", method)
        params = [resource.pop(key) for key in ['method', 'capacity_per_sqkm', 'resource']][0]
        print("resource", resource)
        func = getattr(cutout, params)
        print("func obtained")
        capacity_factor = correction_factor * func(capacity_factor=True, **resource)
        print("cf", capacity_factor)
        layout = capacity_factor * area * cap_per_sqkm
        print("layour", layout)
        profile, capacity = func(shapes=shape, per_unit=True, return_capacity=True, layout=layout, **resource)
        print("capacity", capacity)
        capacities[method] = capacity
        ds = xr.Dataset({
        'profile': profile.rename({'NAME_1': 'bus'}),
        'capacities': capacity.rename({'NAME_1': 'bus'})
        })
        datasets[res] = ds
        ds.to_netcdf(profile_path)

datasets